# STREAMLIT

In [ ]:
!pip install streamlit -q

     |████████████████████████████████| 9.2 MB 6.8 MB/s 
     |████████████████████████████████| 4.7 MB 42.2 MB/s 
     |████████████████████████████████| 182 kB 58.7 MB/s 
     |████████████████████████████████| 237 kB 53.8 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 164 kB 65.4 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 51 kB 5.4 MB/s 


DO THE APP

In [ ]:
%%writefile app.py
import streamlit as st
st.write('# TESTING 🤍🤍🤍🤍🤍🤍')

Writing app.py


In [ ]:
! pip install streamlit==1.13.0 -q

     |████████████████████████████████| 9.2 MB 7.1 MB/s 


RUN APP

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

# RNA MULTICAPA DE 3 CAPAS
 - capa de entrada(N)
 - 1 capa oculta(M)
 - capa de salida(O)

## importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## funcion de activacion

In [ ]:
def func_activacion(n):
  return 1/(1+math.exp(-n))   

In [ ]:
func_activacion_vect = np.vectorize(func_activacion)

## RNA

In [ ]:
def neurona(pesos,entrada, bias):
  prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
  return func_activacion_vect(prod_punto) # f(z)

## RNA Multicapa

In [ ]:
def red_neuronal_multicapa(input, pesosc1, biasc1, pesoscs, biascs):
  # Capa entrada 
  entrada_x = input
  # Capa oculta 1
  salida_c1 = neurona(pesosc1, entrada_x, biasc1)
  # Capa de SALIDA
  output_cs = neurona(pesoscs, salida_c1, biascs)
  # Retornar resultados  
  return salida_c1, output_cs

## derivada de la funcion

In [ ]:
def derivada_f(salida_y): # derivada de la funcion sigmoide
  return salida_y*(1-salida_y)

In [ ]:
def adjust_output(output_wish,size,ExistZero=False):
  SD = np.zeros(size) # vector con el nro de neuronas 
  if ExistZero == True:
    SD[output_wish] = 1
  else:
    SD[output_wish - 1] = 1
  return SD

## backpropagation

In [ ]:
def propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error=0.005,epochs=1000):
  flag = True if 0 in salidas_deseadas.unique() else False
  # longitud de la entrada(caracteristicas)
  n_entradas = entradas.shape[1]
  # Capa oculta 1
  pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biasc1 = 2*np.random.rand(neuronas_c1)-1
  # Capa de salida
  pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 
  biascs = 2*np.random.rand(neuronas_cs) - 1
  # Iterar el algoritmo(nro de epocas)
  for epoch in range(0,epochs):
    for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
      salida_c1, output_cs = red_neuronal_multicapa(entrada_i, pesosc1, biasc1, pesoscs, biascs)
      # ----------------------------------------------------------------------------------------------------
      # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
      # ----------------------------------------------------------------------------------------------------
      salida_deseada = adjust_output(salida_deseada_i,neuronas_cs,flag)
      # Calcular Δ para la CAPA de SALIDA(cs): 
      delta_cs = derivada_f(output_cs)*(salida_deseada - output_cs)
      peso_anterior_cs = pesoscs
      # Actualizar pesos que llegan a la capa de SALIDA
      pesoscs = pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
      biascs = biascs + alfa * delta_cs*1   
      # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
      delta_c1 = derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
      # Actualizar pesos  que entran a las neuronas de la capa oculta 1
      pesosc1 = pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
      biasc1 = biasc1 + alfa * delta_c1 * 1  

  return pesosc1,biasc1,pesoscs,biascs 

## fit RNA

In [ ]:
def fit_rna(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs):
  pesosc1,biasc1,pesoscs,biascs = propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs)
  return pesosc1,biasc1,pesoscs,biascs

## predict RNA

In [ ]:
def predict_rna(x_test, valores_obtenidos):
  pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
  y_predict_list = []
  for entrada in x_test:
    salida_c1, output_cs = red_neuronal_multicapa(entrada, pesosc1,biasc1,pesoscs,biascs)
    y_predict_list.append(output_cs)
  return y_predict_list

# NRA OOP

In [ ]:
%%writefile MultilayerPerceptron.py

import numpy as np
import math
from sklearn import preprocessing
import pandas as pd

class MultilayerPerceptron:
    # ======================================================================================================================== 
    def __init__(self,nro_entradas_x,neuronas_c1,neuronas_cs):
        #self.n_entradas = entradas.shape[1]
        self.neuronas_c1 = neuronas_c1
        self.neuronas_cs = neuronas_cs
        # Capa oculta 1
        self.pesosc1 = 2*np.random.rand(self.neuronas_c1,nro_entradas_x) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
        self.biasc1 = 2*np.random.rand(self.neuronas_c1) - 1
        # Capa de salida
        self.pesoscs = 2*np.random.rand(self.neuronas_cs,self.neuronas_c1) - 1 
        self.biascs = 2*np.random.rand(self.neuronas_cs) - 1

        self.pesos_bias_optimos = [self.pesosc1,self.biasc1,self.pesoscs,self.biascs]
        
        self.scaler = preprocessing.StandardScaler()

    # ======================================================================================================================== 
    def func_activacion(self, n):
        return 1/(1+math.exp(-n))
    # ======================================================================================================================== 
    def neurona(self, pesos,entrada, bias):
        self.func_activacion_vect = np.vectorize(self.func_activacion)
        prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
        return self.func_activacion_vect(prod_punto) # f(z)
    # ======================================================================================================================== 
    def red_neuronal_multicapa(self, input, pesosc1, biasc1, pesoscs, biascs):
        # Capa entrada 
        entrada_x = input
        # Capa oculta 1
        self.salida_c1 = self.neurona(pesosc1, entrada_x, biasc1)
        # Capa de SALIDA
        self.output_cs = self.neurona(pesoscs, self.salida_c1, biascs)
        # Retornar resultados  
        return self.salida_c1, self.output_cs
    # ======================================================================================================================== 
    def derivada_f(self, salida_y): # derivada de la funcion sigmoide
        return salida_y*(1-salida_y)
    # ======================================================================================================================== 
    def adjust_output(self, output_wish,size,ExistZero=False):
        self.SD = np.zeros(size) # vector con el nro de neuronas 
        if ExistZero == True:
          self.SD[output_wish] = 1
        else:
          self.SD[output_wish - 1] = 1
        return self.SD
    # ======================================================================================================================== 
    def propagacion_hacia_atras(self, entradas,salidas_deseadas,alfa,epochs=1000):
        flag = True if 0 in salidas_deseadas.unique() else False
        #nro_neuronas_cs = len(salidas_deseadas.unique())
        # longitud de la entrada(caracteristicas)
        n_entradas = entradas.shape[1]
        """
        # Capa oculta 1
        pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
        biasc1 = 2*np.random.rand(neuronas_c1)-1
        # Capa de salida
        pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 
        biascs = 2*np.random.rand(neuronas_cs) - 1
        """
        # Iterar el algoritmo(nro de epocas)
        for epoch in range(0,epochs):
          for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
            salida_c1, output_cs = self.red_neuronal_multicapa(entrada_i, self.pesosc1, self.biasc1, self.pesoscs, self.biascs)
            # ----------------------------------------------------------------------------------------------------
            # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
            # ----------------------------------------------------------------------------------------------------
            salida_deseada = self.adjust_output(salida_deseada_i, self.neuronas_cs ,flag)
            # Calcular Δ para la CAPA de SALIDA(cs): 
            delta_cs = self.derivada_f(output_cs)*(salida_deseada - output_cs)
            peso_anterior_cs = self.pesoscs
            # Actualizar pesos que llegan a la capa de SALIDA
            self.pesoscs = self.pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
            self.biascs = self.biascs + alfa * delta_cs   
            # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
            delta_c1 = self.derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
            # Actualizar pesos  que entran a las neuronas de la capa oculta 1
            self.pesosc1 = self.pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
            self.biasc1 = self.biasc1 + alfa * delta_c1  
        return self.pesosc1, self.biasc1, self.pesoscs, self.biascs 

    # ======================================================================================================================== 
    def fit_rna(self, entradas,salidas_deseadas,alfa,epochs):
        self.pesosc1,self.biasc1,self.pesoscs,self.biascs = self.propagacion_hacia_atras(entradas,salidas_deseadas,alfa,epochs)
        return self.pesosc1,self.biasc1,self.pesoscs,self.biascs

    # ========================================================================================================================  
    # =====================  ajustar salida ================ 
    def ajuste(self, valores_pred):
        result_pred = []
        for vector in valores_pred:
          indice = np.argmax(vector)
          result_pred.append(indice+1)
        self.y_pred = pd.Series(result_pred)
        return self.y_pred

    # ========================================================================================================================  
    def predict_rna(self, x_test):
        #pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
        y_predict_list = []
        for entrada in x_test:
          salida_c1, output_cs = self.red_neuronal_multicapa(entrada, self.pesosc1, self.biasc1, self.pesoscs, self.biascs)
          y_predict_list.append(output_cs)
        self.y_pred = self.ajuste(y_predict_list)
        return self.y_pred

    def normalizacion(self, X):
        # =========== NORMALIZANDO =========== 
        self.scaler.fit(X)
        # =========== TRANSFORMANDO =========== 
        self.X = self.scaler.transform(X)
        return self.X


Overwriting MultilayerPerceptron.py


# DETECCION DE VINO CON RNA

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv("wine.csv")
df.head()

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280_OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# ++++++++++ PROBANDO LA CLASE CREADA
MLP = MultilayerPerceptron(13,28,3)

In [ ]:
# ++++++++++ PROBANDO LA CLASE CREADA
X = df.drop("class", axis=1)
Y = df["class"]
X = MLP.normalizacion(X)

In [ ]:
# =========== SEPARACION DE DATOS =========== 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

## modelamiento

In [ ]:
# ++++++++++ PROBANDO LA CLASE CREADA
weigths = MLP.fit_rna(X_train,y_train,0.001,150)

In [ ]:
# ++++++++++ PROBANDO LA CLASE CREADA
y_pred = MLP.predict_rna(X_test)

## metricas

In [ ]:
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       1.00      0.93      0.96        14
           3       0.89      1.00      0.94         8

    accuracy                           0.97        36
   macro avg       0.96      0.98      0.97        36
weighted avg       0.98      0.97      0.97        36



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score, f1_score

#verificando generalización
print("Prueba - Generalización")
print(balanced_accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="weighted"))

Prueba - Generalización
0.9761904761904763
0.9725248123940935


## GRABAR MODELO

In [ ]:
model = MLP

In [ ]:
import pickle

grabar modelo

In [ ]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename,'wb') as file:
  pickle.dump(model,file)

cargar modelo

In [ ]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename,'rb') as file:
  model = pickle.load(file)

# testear que se haya recuperado correctamente los pesos

In [ ]:
y_pred = model.predict_rna(X_test)

In [ ]:
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       1.00      0.93      0.96        14
           3       0.89      1.00      0.94         8

    accuracy                           0.97        36
   macro avg       0.96      0.98      0.97        36
weighted avg       0.98      0.97      0.97        36



## APP prediccion

In [ ]:
# https://www.youtube.com/watch?v=vibDbEBnyV4

In [ ]:
%%writefile app.py

from sklearn import preprocessing
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
from MultilayerPerceptron import *



# PATH DEL MODELO ENTRENADO
MODEL_PATH = '/content/pickle_model.pkl'

# se recibe la imagen y el modelo devuelve la prediccion
def model_prediccion(entrada, modelo):
  x = np.asarray(entrada).reshape(1,-1)

  # NORMALIZANDO LOS DATOS DE ENTRADA
  entrada_df = np.array([entrada]) # convertir a array de numpy y lista de listas
  entrada_df = modelo.scaler.transform(entrada_df)

  # predecir
  preds = modelo.predict_rna(entrada_df)
  return preds


def main():

  #model = ''
  # cargar el modelo
  with open(MODEL_PATH, 'rb') as file:
    model = pickle.load(file)

  # titulo
  html_temp = """
  <h1 style="color:#008000;text-align:center;">SISTEMA DE DETECCION DE LA CALIDAD DE VINOS </h1>
  </div>
  """
  st.markdown(html_temp, unsafe_allow_html=True)

  # lectura de datos
  
  d1 = st.text_input("Alcohol:")
  d2 = st.text_input("Malic acid:")
  d3 = st.text_input("Ash:")
  d4 = st.text_input("Alcalinity of ash:")
  d5 = st.text_input("Magnesium:")
  d6 = st.text_input("Total phenols:")
  d7 = st.text_input("Flavanoids:")
  d8 = st.text_input("Nonflavanoid phenols:")
  d9 = st.text_input("Proanthocyanins:")
  d10 = st.text_input("Color intensity:")
  d11 = st.text_input("Hue:")
  d12 = st.text_input("OD280_OD315:")
  d13 = st.text_input("Proline:")

  # el boton prediccion se usa para iniciar el procesamiento
  if st.button("PREDECIR"):
    x_in = [np.float_(d1.title()),
            np.float_(d2.title()),
            np.float_(d3.title()),
            np.float_(d4.title()),
            np.float_(d5.title()),
            np.float_(d6.title()),
            np.float_(d7.title()),
            np.float_(d8.title()),
            np.float_(d9.title()),
            np.float_(d10.title()),
            np.float_(d11.title()),
            np.float_(d12.title()),
            np.float_(d13.title())  ]
    predictWine = model_prediccion(x_in, model)
    if predictWine[0] == 1:
      pred = "MALA"
    elif predictWine[0] == 2:
      pred = "BUENA"
    else:
      pred = "EXCELENTE"
    st.success("EL VINO TIENE UNA CALIDAD: {}".format(pred))

if __name__ == '__main__':
  main()



Overwriting app.py


# ejecutar programa

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-12-23 03:55:08.326 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.951s



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.230.10.231:8501

your url is: https://fair-areas-watch-35-230-10-231.loca.lt
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  wa

In [ ]:
entrada = leer_datos()

insert value of Alcohol: 15
insert value of Malic acid: 1.8
insert value of Ash: 2.55
insert value of Alcalinity of ash: 14
insert value of Magnesium: 120
insert value of Total phenols: 2.70
insert value of Flavanoids: 3.01
insert value of Nonflavanoid phenols: 0.33
insert value of Proanthocyanins: 2.35
insert value of Color intensity: 5.55
insert value of Hue: 1.09
insert value of OD280_OD315: 3.55
insert value of Proline: 1060


In [ ]:
result = np.argmax(predict_rna(entrada_df, pesos_bias_optimos))
if result == 0:
  print("CALIDAD BAJA")
elif result == 1 :
  print("CALIDAD REGULAR")
else:
  print("CALIDAD ALTA")

CALIDAD BAJA
